In [ ]:
%matplotlib qt
from __future__ import print_function
from IPython.display import HTML
from dolfin import *
from dolfin_adjoint import *
import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import numpy as np
import copy
import scipy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import heapq
#plotly.tools.set_credentials_file(username='Fczetndh1', api_key='6apj938i0WVA2e9qwYr8')
#import pyipopt

In [ ]:
# this ensures that the parameters calculated by the PDE stay in the same index as the mesh coordinates
parameters["reorder_dofs_serial"] = False

#Using an nxn square mesh and S total turned on source terms
S = Constant(5)
# I will use a 32 x 32 mesh with 16 finite elements
m = 32
n = 8
mesh = UnitSquareMesh(m, m)

# A piecewise linear function space
V = FunctionSpace(mesh, "CG", 1) 

# Dirichlet boundary condition
def boundary(x):
    return x[0] < DOLFIN_EPS or x[0] > 1 - DOLFIN_EPS or x[1] < DOLFIN_EPS or x[1] > 1 - DOLFIN_EPS

u0 = Constant(0.0)
bc = DirichletBC(V, u0, boundary)

# let a = 100, sig^2 = .02
# Create Gaussian Source Term
sigma = np.sqrt((1.0/(n**2)) / 6.25)  # parameters follow mintOC

def source_term(xk,yk, w, sigma, degree, Fspace):
    return interpolate(Expression('w*100*exp(-((pow(x[0] - xk,2) + pow(x[1] - yk,2)) / pow(sigma,2)))', w=w, xk=xk, yk=yk, sigma=sigma, degree=degree), Fspace)

In [ ]:
# First I will solve for a simple reference solution, and then two complex one

# simple reference function
xbar = .1
ybar = .4
f = source_term(xbar, ybar, 1, sigma, 1, V)

u_ref = Function(V) # reference solution
v1 = TestFunction(V)  # test function

# Solve the forward PDE
F_ref = (inner(grad(v1), grad(u_ref)) - f*v1)*dx
solve(F_ref == 0, u_ref, bc)

# complex reference function
xbar1 = .8
ybar1 = .8

xbar2 = .9
ybar2 = .2

fb = source_term(xbar, ybar, 1, sigma, 1, V)
f1 = source_term(xbar1, ybar1, 1, sigma, 1, V)
f2 = source_term(xbar2, ybar2, 1, sigma, 1, V)

# the complex reference function
f1.vector()[:] += fb.vector() + f2.vector()

u_refc = Function(V) 
vc = TestFunction(V) 


# Solve the forward PDE for the complex function
F_refc = (inner(grad(vc), grad(u_refc)) - f1*vc)*dx
solve(F_refc == 0, u_refc, bc)


#complex reference function 2
fv = source_term(.1, .9, 1, sigma, 1, V)
f1v = source_term(.1, .1, 1, sigma, 1, V)

# the complex reference function
fv.vector()[:] += f1v.vector() 

u_3 = Function(V) # reference solution
v3 = TestFunction(V)  # test function

# Solve the forward PDE for the complex function
F3 = (inner(grad(v3), grad(u_3)) - fv*v3)*dx
solve(F3 == 0, u_3, bc)

In [ ]:
l = np.linspace(0,1,m+1)
X,Y = np.meshgrid(l,l)
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('Source Function Value')
ax.set_title('PDE Solution to Simple Reference Function')
ax.plot_surface(X, Y, u_ref.vector().array().reshape((m+1,m+1)), cmap='viridis', linewidth=0)
plt.show()


fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('Source Function Value')
ax.set_title('PDE Solution to Complex Reference Function')
ax.plot_surface(X, Y, u_refc.vector().array().reshape((m+1,m+1)), cmap='viridis', linewidth=0)
plt.show()


fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('Source Function Value')
ax.set_title('PDE Solution to Complex Reference Function 2')
ax.plot_surface(X, Y, u_3.vector().array().reshape((m+1,m+1)), cmap='viridis', linewidth=0)
plt.show()

In [ ]:
l = np.linspace(0,1,m+1)
X,Y = np.meshgrid(l,l)
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('Reference Function Value')
ax.set_title('Actual Simple Reference Function')
ax.plot_surface(X, Y, f.vector().array().reshape((m+1,m+1)), cmap='viridis', linewidth=0)
plt.show()


fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('Reference Function Value')
ax.set_title('Actual Complex Reference Function')
ax.plot_surface(X, Y, f1.vector().array().reshape((m+1,m+1)), cmap='viridis', linewidth=0)
plt.show()


fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('Reference  Function Value')
ax.set_title('Actual Complex Reference Function 2')
ax.plot_surface(X, Y, fv.vector().array().reshape((m+1,m+1)), cmap='viridis', linewidth=0)
plt.show()

In [ ]:
# u_ref is the reference solution. I want to minimize the distance between this reference solution and the sum of a bunch of basis functions
# (source terms) over the mesh. Ideally, this should turn on functions near (xbar1, ybar1) and (xbar2, ybar2)

'''
Each source term is normalized by dividing by the total number of source terms.
This ensures that the total source term f_kl at each mesh location (which is the sum
of all (n+1)^2 source terms evaluated at that mesh point) has the same scale as the
reference solution/
'''

coords = np.linspace(0, 1, n+1)[:-1]
coords += 0.5 * coords[1]
clist = []
for a in coords:
    for b in coords:
        clist.append([a,b])
        
    
scale_factor = 1.0 / (n**2)     
source_expression_list = [source_term(xk, yk, scale_factor, sigma, 1, V) for xk, yk in clist]

# Each term in the list is the value of a basis function over all points on the mesh
constraint_list = []
for t in source_expression_list:   
    constraint_list.append(t.vector().array())


wHat = source_expression_list[0]
for elem in source_expression_list[1:]:
    wHat.vector()[:] += elem.vector()


# Each row of the constraint matrix is the sum of weak element terms at location kl
# This is M^2 x N  -- noVertices x noSourceTerms
Constraint_matrix = np.array(np.array(constraint_list).transpose())   

# Trial and Test Functions for weak element formulation
u = Function(V)
v = TestFunction(V)  

# Solve the forward PDE for the solution vector
# note that this solution is not dependent on the reference function
F = (inner(grad(v), grad(u)) - wHat*v)*dx
solve(F == 0, u, bc)

In [ ]:
# the sum of all source terms evaluated across the mesh
fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('Source Function Value')
ax.set_title('Sum of Vertex Source Terms over Mesh')
ax.plot_surface(X, Y, wHat.vector().array().reshape((m+1,m+1)), cmap='viridis', linewidth=0)
plt.show()

ax = fig.add_subplot(122, projection='3d')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('Source Function Value')
ax.set_title('PDE Solution to Source Term Functions')
ax.plot_surface(X, Y, u.vector().array().reshape((m+1,m+1)), cmap='viridis', linewidth=0)
plt.show()

In [ ]:
# The functional evaluates the difference between the discretized reference function and the sum of all activated source terms over the entire mesh
# simple reference function case
J = Functional((0.5*inner(u-u_ref, u-u_ref))*dx)

# complex reference function case
J_complex = Functional((0.5*inner(u-u_refc, u-u_refc))*dx) 

# complex reference 2
J_cv = Functional((0.5*inner(u-u_3, u-u_3))*dx) 

'''
The value of the source terms at each mesh vertex are the controls. Intuitively, I want to 
"turn on" source terms near the reference solution, and "turn off" all other source source terms,
so I can approximate the reference function with a sum of gaussian bump source functions that are
centered near the reference function's (a (sum of gaussian) bump(s) itself) location, which is not
on a mesh vertex, but rather somewhere between mesh vertices.
'''

# I make the source terms into a control
w_control = FunctionControl(wHat)

# the reduced functional is a function of the source terms
rf = ReducedFunctional(J, w_control)
rf_complex = ReducedFunctional(J_complex, w_control)
rf_cv =  ReducedFunctional(J_cv, w_control)

In [ ]:
# bounds for each of the decision variables (ie: no upper bound)
lb = 0.0
ub = 1000000000.0

# callback function for the parameters (ie: locations of w_kl)
def iter_cb(w):
    print("w = ", w)
    return

# Find the optimal value of the source terms at each vertex
m_opt = minimize(rf, method='SLSQP', bounds=(lb, ub), options={'ftol': 1e-9, 'maxiter' : 100, 'disp': True})
m_opt2 = minimize(rf_complex, method='SLSQP', bounds=(lb, ub), callback=iter_cb, options={'ftol': 1e-9, 'maxiter' : 100, 'disp': True})
m_opt3 = minimize(rf_cv, method='SLSQP', bounds=(lb, ub), callback=iter_cb, options={'ftol': 1e-9, 'maxiter' : 100, 'disp': True})

# These are the optimal values of the sum of all basis functions at each vertex on the grid
# Now I need to isolate the coefficient values for each w_kl
w_star = m_opt.vector().array()
w_star_complex = m_opt2.vector().array()
w_cv = m_opt3.vector().array()

In [ ]:
# The upper bound on the sum of all coefficients (continuous) and max number of activated sources (integer)
S = 4.0
x_vec = np.ones(n**2)

# bounds on each w_kl coefficient - continuous solution
bds =[(0, 1)]*len(x_vec)

# objective function to find optimal w_kl values for each reference solution by OLS
def func(x):
    return sum(((np.dot(Constraint_matrix, x) - w_star)**2))

def func1(x):
    return sum(((np.dot(Constraint_matrix, x) - w_star_complex)**2))

def func2(x):
    return sum(((np.dot(Constraint_matrix, x) - w_cv)**2))

# continuous constraint to place an upper bound on coefficient values
cons = {'type' : 'ineq', 'fun': lambda x: -sum(x) + S}

# integer constraint to active a maximum of S source - this does not work
cons_integer =({'type': 'eq', 'fun': lambda x: x - x**2}, {'type': 'ineq', 'fun': lambda x: S-sum(x)})
    
res_simple = scipy.optimize.minimize(func, x_vec, method='SLSQP', bounds=bds, constraints=cons, callback=None, options={'ftol': 1e-9, 'maxiter' : 200, 'disp': True})
res_complex = scipy.optimize.minimize(func1, x_vec, method='SLSQP', bounds=bds, constraints=cons, callback=None, options={'ftol': 1e-9, 'maxiter' : 200, 'disp': True})
res_cv = scipy.optimize.minimize(func2, x_vec, method='SLSQP', bounds=bds, constraints=cons, callback=None, options={'ftol': 1e-9, 'maxiter' : 200, 'disp': True})

#res_simple_integer = scipy.optimize.minimize(func, rvec, method='SLSQP', bounds=bds, constraints=cons_integer, callback=None, options={'ftol': 1e-9, 'maxiter' : 25, 'disp': True})
#res_complex_integer = scipy.optimize.minimize(func1, x_vec, method='SLSQP', constraints=cons_integer, callback=None, options={'ftol': 1e-9, 'maxiter' : 25, 'disp': True})
#res_cv_integer = scipy.optimize.minimize(func2, x_vec, method='SLSQP', constraints=cons_integer, callback=None, options={'ftol': 1e-9, 'maxiter' : 25, 'disp': True})

In [ ]:
# This function will overlay the the kxk grid of source parameters into the mesh for n= 8, m = 32
def convertGrid832(source_params):
    grid = np.zeros((m+1, m+1))
    count = 0
    for a in range(2, m+1, 4):
        for b in range(2, m+1, 4):
            grid[a][b] = source_params[count]
            count += 1
    return(grid)

In [ ]:
# Manually extracting the optimal parameter values at each vertex
r_simple = convertGrid832(res_simple.x)
r_complex = convertGrid832(res_complex.x)
r_cv = convertGrid832(res_cv.x)
intS = int(S)

# for integer solutions turn on the largest S terms
m1 = heapq.nlargest(intS, r_simple.flatten())
m2 = heapq.nlargest(intS, r_complex.flatten())
m3 = heapq.nlargest(intS, r_cv.flatten())

# integer solutions for each case
r_complex_int = copy.deepcopy(r_complex)
r_complex_int[r_complex < m2[-1]] = 0
r_complex_int[r_complex_int !=0] = 1
r_cv_int = copy.deepcopy(r_cv)
r_cv_int[r_cv < m3[-1]] = 0
r_cv_int[r_cv_int != 0] = 1
r_simple_int = copy.deepcopy(r_simple)
r_simple_int[r_simple < m1[-1]] = 0
r_simple_int[r_simple_int != 0] = 1

fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
ax.plot_surface(X, Y, r_simple, cmap='viridis', linewidth=0)
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('value of w_{xy}')
ax.set_title('Optimal Parameter Values - Simple Reference Continuous')
plt.show()


ax = fig.add_subplot(122, projection='3d')
ax.plot_surface(X, Y, r_simple_int, cmap='viridis', linewidth=0)
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('value of w_{xy}')
ax.set_title('Optimal Parameter Values - Simple Reference Integer')
plt.show()


# Manually extracting the optimal parameter values at each vertex - complex reference function
fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
ax.plot_surface(X, Y, r_complex, cmap='viridis', linewidth=0)
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('value of w_{xy}')
ax.set_title('Optimal Parameter Values - Complex Reference Continuous')
plt.show()


ax = fig.add_subplot(122, projection='3d')
ax.plot_surface(X, Y, r_complex_int, cmap='viridis', linewidth=0)
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('value of w_{xy}')
ax.set_title('Optimal Parameter Values - Complex Reference Integer')
plt.show()


# Complex Reference Function 2
fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
ax.plot_surface(X, Y, r_cv, cmap='viridis', linewidth=0)
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('value of w_{xy}')
ax.set_title('Optimal Parameter Values - Complex Reference Continuous 2')
plt.show()

ax = fig.add_subplot(122, projection='3d')
ax.plot_surface(X, Y, r_cv_int, cmap='viridis', linewidth=0)
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('value of w_{xy}')
ax.set_title('Optimal Parameter Values - Complex Reference Integer 2')
plt.show()

In [ ]:
w_list= []
for a in range(len(clist)):
    w_list.append([clist[a][0], clist[a][1], res_complex.x[a]])
print(w_list)
example_list = [source_term(xk, yk, w_k*scale_factor, sigma, 1, V) for xk, yk, w_k in w_list]

# Each term in the list is the value of a basis function over all points on the mesh

approx = example_list[0]
for elem in example_list[1:]:
    approx.vector()[:] += elem.vector()
    
# the sum of all source terms evaluated across the mesh
fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('Source Function Value')
ax.set_title('An Unscaled Approximation')
ax.plot_surface(X, Y, approx.vector().array().reshape((m+1,m+1)), cmap='viridis', linewidth=0)

ax = fig.add_subplot(122, projection='3d')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_zlabel('Reference Function Value')
ax.set_title('PDE Solution to Complex Reference Function')
ax.plot_surface(X, Y, u_refc.vector().array().reshape((m+1,m+1)), cmap='viridis', linewidth=0)
plt.show()
plt.show()

In [ ]:
# Error


# find the 3 errors

error_simple = u_ref.vector().array()

In [ ]:
X = []
Y = []
Z = []
for a in range(len(clist)):
    X.append(clist[a][0])
    Y.append(clist[a][1])
    Z.append(res_simple.x[a])
    
z_max = max(res_simple.x)


trace1 = go.Scatter3d(
    x=X,
    y=Y,
    z=Z,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.2
        ),
        opacity=0.8
    )
)

trace2 = go.Scatter3d(
    x=xbar,
    y=ybar,
    z=z_max + .1*z_max,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgb(204, 204, 204)',
            width=.4
        ),
        opacity=0.9
    )
)

data = [trace1, trace2]

layout = go.Layout(
    title='Unnormalized Optimal Parameter Values - Simple Reference',
    scene = go.Scene(
    xaxis=dict(
        title='X Axis',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Y Axis',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    zaxis=dict(
        title='w_xy',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
    ) 
    
)

fig = go.Figure(data=data, layout=layout)
plotly.plotly.iplot(fig, filename='simple_params')

In [ ]:
X = []
Y = []
Z = []
for a in range(len(clist)):
    X.append(clist[a][0])
    Y.append(clist[a][1])
    Z.append(res_complex.x[a])
    
z_max = max(res_complex.x) + .1*max(res_complex.x)

x1 = [xbar, xbar1, xbar2]
y1 = [ybar, ybar1, ybar2]   
z1 = [z_max, z_max, z_max]


trace1 = go.Scatter3d(
    x=X,
    y=Y,
    z=Z,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.2
        ),
        opacity=0.8
    )
)

trace2 = go.Scatter3d(
    x=x1,
    y=y1,
    z=z1,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgb(204, 204, 204)',
            width=.4
        ),
        opacity=0.9
    )
)

data = [trace1, trace2]

layout = go.Layout(
    title='Unnormalized Optimal Parameter Values - Complex Reference',
    scene = go.Scene(
    xaxis=dict(
        title='X Axis',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Y Axis',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    zaxis=dict(
        title='w_xy',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
    ) 
    
)

fig = go.Figure(data=data, layout=layout)
plotly.plotly.iplot(fig, filename='complex_params')

In [ ]:
X = []
Y = []
Z = []
for a in range(len(clist)):
    X.append(clist[a][0])
    Y.append(clist[a][1])
    Z.append(res_cv.x[a])

z_max = max(res_cv.x) + .1*max(res_cv.x)

x1 = [.1, .1]
y1 = [.7, .1]  

z1 = [z_max, z_max]
    


trace1 = go.Scatter3d(
    x=X,
    y=Y,
    z=Z,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.2
        ),
        opacity=0.8
    )
)

trace2 = go.Scatter3d(
    x=x1,
    y=y1,
    z=z1,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgb(204, 204, 204)',
            width=.4
        ),
        opacity=0.9
    )
)

data = [trace1, trace2]

layout = go.Layout(
    title='Unnormalized Optimal Parameter Values - Complex Reference',
    scene = go.Scene(
    xaxis=dict(
        title='X Axis',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Y Axis',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    zaxis=dict(
        title='w_xy',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
    ) 
    
)

fig = go.Figure(data=data, layout=layout)
plotly.plotly.iplot(fig, filename='complex_params_lowvar')